<h1> Do the presence of certain venues increase the spread of Covid-19 </h1>

<h3> Introduction/Background </h3>

In this project I would like to look at how the social environment may effect the spread of Covid-19. This project will primarly look at different venue types and their potential affect on Covid-19 spread. We know that social-distancing is important to help prevent the spread of covid-19 and we also know that certain business venues allow for different levels of distancing. For example, the purpose of a bar is to interact with other persons, so even while "practicing" social distancing, a bar is probably more likely to spread Covid-19 verus a venue where the primary purpose isn't interaction; e.g. a grocery store. All of the different zipcode areas within Manhattan and their surrounding venues in comparrison with Covid-19 death rates will be used as the subject of this project.


<h3> Reasoning/Assumptions </h3>

The <b>'reservations'</b> section will discuss why our assumptions are likely invalid.

* We specifically chose to only include Manhattan instead of all of New York City (or the other parts of country) to try and limit the geographical effect on viral transmission within this project. 

* We have used up to date death rates even though the effect of venue presence would have been muted once lockdowns were put into place. We assume that the original damedge was done early before lockdowns were put into place and that the current death rates are a carry forward effect from when the venues were still open.

* We have used death rate instead of case rate because of the amount of cases that go unreported. We assume it is much less likely to fail to report/attribute a death, than it is to fail to report a case (e.g. someone is assymtomatic and never gets tested.)

<h3> Data Collection </h3>

The following github link was used to gather Manhattan zipcode data as well as the associated death rates for those areas:

https://github.com/nychealth/coronavirus-data/blob/master/data-by-modzcta.csv

We also needed data for the venues which surround each zipcode area. To get this data we used the FourSquare.com explore endpoint API. The API returns a category for each venue so that two differently named bars will both return as "bar". This allows us to group the prevenelce of each venue type to each area.

<h3> Modeling </h3>

There will be two different types of models eployed. The first will be a Multiple Linear Regression model to try and estimate the covid-19 death rate based off of the data set. We do not expect this model to perform well for reasons which are listed under the <b>'reservations'</b> section.

The second model will use K-Means to cluster similar zipcode regions based entirely off of venue type. We will then compare the average death rate against the different clusters to see if there is a difference between clusters. ANOVA may be a suitable test to check if there is a significant difference of means. I would like to add that this model will also come with many reservations which will be listed in the 'reservations' section.

<h3> Purpose and Use </h3>

The purpose of this model is to see if the presence of certain venues lead to an increase covid-19 transmission. If it were possible to make such a determination, that information would be very valuable to government officials deciding how to re-open their economies. 

<h3> Reservations/Issues </h3>

* People who die within one zip code area may have been originally infected from a venue in a different area.

* Certain venues may attract different demographics. It is these demographics which may more likely be the root instead of the venues themselves. For example bars may attract young people so an area with a lot of bars might have a demographically lower median age which could lead to less deaths. On the other side of that argument, certain venues may attract demographics that are more/less likely to practice proper social distancing.

* One population center may be more likely to have increased death rates if it is adjecent to another area with high death rates regardless of which venues it contains.

* The data fails to look at: population density of different areas; venue density; or venue count. <b>It rather only identifies venue prevelence relative to its own area</b>.



<h3> Imports </h3>

In [35]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

print('Libraries imported.')

ModuleNotFoundError: No module named 'folium'

<h3> Beautiful Soup used to scrap data</h3>

In [51]:
import pandas as pd
from bs4 import BeautifulSoup
import urllib.request

url = 'https://github.com/nychealth/coronavirus-data/blob/master/data-by-modzcta.csv'
fp = urllib.request.urlopen(url)
mybytes = fp.read()

mystr = mybytes.decode("utf8")
fp.close()
soup = BeautifulSoup(mystr, 'html.parser')

table = soup.find_all('table')[0] 
df = pd.read_html(str(table))
df=df[0]
df = df[df['BOROUGH_GROUP']== 'Manhattan']
df1 = df[['NEIGHBORHOOD_NAME','MODIFIED_ZCTA','COVID_DEATH_RATE']]
df1.head()

,NEIGHBORHOOD_NAME,MODIFIED_ZCTA,COVID_DEATH_RATE
0,Chelsea/NoMad/West Chelsea,10001,97.61
1,Chinatown/Lower East Side,10002,200.64
2,East Village/Gramercy/Greenwich Village,10003,61.34
3,Financial District,10004,27.39
4,Financial District,10005,23.82


<h3> FourSquare API Explore endpoint used to collect venue data associated with zipcode areas</h3>

In [96]:
CLIENT_ID = 'VI1AEMLO03KXWQOYOSRPBFFT40HJOQ1ZYFXUC2HFJXUVAAUC' # your Foursquare ID
CLIENT_SECRET = 'MALO04YAJ1XF3JBRX4TSBHKQJTDYMQOCJMGQOEW3XA0GWOOJ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)
LIMIT = 100
def getNearbyVenues(names, zipcode, radius=250):
    
    venues_list=[]
    for name, zipcode in zip(names, zipcode):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near={}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            zipcode, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            zipcode, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'zipcode',
                  'Venue Name',
                  'Lat coord',
                  'Long coord',
                  'Venue Category']
    
    return(nearby_venues)


Your credentails:
CLIENT_ID: VI1AEMLO03KXWQOYOSRPBFFT40HJOQ1ZYFXUC2HFJXUVAAUC
CLIENT_SECRET:MALO04YAJ1XF3JBRX4TSBHKQJTDYMQOCJMGQOEW3XA0GWOOJ


In [97]:
manhattan_venues = getNearbyVenues(names=df1['NEIGHBORHOOD_NAME'],
                                   zipcode=df1['MODIFIED_ZCTA'],
                                  )

Chelsea/NoMad/West Chelsea
Chinatown/Lower East Side
East Village/Gramercy/Greenwich Village
Financial District
Financial District
Financial District
TriBeCa
Alphabet City/East Village/Stuyvesant Town-Cooper Village
Flatiron/Gramercy/Kips Bay
Chelsea
Greenwich Village/SoHo
Hudson Square/Little Italy/SoHo/TriBeCa
Hudson Square/Meatpacking District/West Village
Kips Bay/Murray Hill/NoMad
East Midtown/Murray Hill
Hell's Kitchen/Midtown Manhattan
Hell's Kitchen/Midtown Manhattan
Lenox Hill/Upper East Side
East Midtown
Lincoln Square
Upper West Side
Manhattan Valley/Morningside Heights/Upper West Side
Central Harlem (South)
Central Harlem (South)/Morningside Heights/West Harlem
Upper East Side/Yorkville
East Harlem
Central Harlem (North)
Hamilton Heights/West Harlem
Washington Heights (South)
Washington Heights (North)/Washington Heights (South)
Inwood/Washington Heights (North)
East Harlem
Hell's Kitchen/Midtown Manhattan
Central Harlem (North)/East Harlem
Chinatown/Financial District
Cent

In [101]:
manhattan_venues.head()
manhattan_venues.groupby('zipcode').count()

,Neighborhood,Venue Name,Lat coord,Long coord,Venue Category
zipcode,,,,,
10001,39,39,39,39,39
10002,39,39,39,39,39
10003,25,25,25,25,25
10004,8,8,8,8,8
10005,51,51,51,51,51
10006,38,38,38,38,38
10007,40,40,40,40,40
10009,35,35,35,35,35
10010,31,31,31,31,31


In [102]:
# one hot encoding
manhattan_onehot = pd.get_dummies(manhattan_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['zipcode'] = manhattan_venues['zipcode'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_grouped = manhattan_onehot.groupby('zipcode').mean().reset_index()
manhattan_grouped

,zipcode,Accessories Store,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Stop,Butcher,Café,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,College Bookstore,College Theater,Comedy Club,Comfort Food Restaurant,Community Center,Concert Hall,Convenience Store,Cooking School,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Flea Market,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,High School,Historic Site,History Museum,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Martial Arts Dojo,Mediterranean Restaurant,Memorial Site,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Optical Shop,Outdoors & Recreation,Paella Restaurant,Pakistani Restaurant,Park,Performing Arts Venue,Perfume Shop,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Physical Therapist,Pie Shop,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pub,Public Art,Ramen Restaurant,Record Shop,Rental Car Location,Rest Area,Restaurant,River,Rock Climbing Spot,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,School,Seafood Restaurant,Shipping Store,Shoe Store,Smoke Shop,Snack Place,Soba Restaurant,Social Club,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Storage Facility,Street Art,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tapas Restaurant,Tattoo Parlor,Tea Room,Temple,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Thrift / Vintage Store,Tiki Bar,Toy / Game Store,Trail,Tree,Turkish Restaurant,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Vietnamese Restaurant,Watch Shop,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,10001,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000

<h3> Merges death rate from the 

In [107]:
final_data = manhattan_grouped.merge(df1,left_on='zipcode',right_on='MODIFIED_ZCTA',how='inner')
final_data.head()

,zipcode,Accessories Store,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Stop,Butcher,Café,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,College Bookstore,College Theater,Comedy Club,Comfort Food Restaurant,Community Center,Concert Hall,Convenience Store,Cooking School,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Flea Market,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,High School,Historic Site,History Museum,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Martial Arts Dojo,Mediterranean Restaurant,Memorial Site,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Optical Shop,Outdoors & Recreation,Paella Restaurant,Pakistani Restaurant,Park,Performing Arts Venue,Perfume Shop,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Physical Therapist,Pie Shop,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pub,Public Art,Ramen Restaurant,Record Shop,Rental Car Location,Rest Area,Restaurant,River,Rock Climbing Spot,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,School,Seafood Restaurant,Shipping Store,Shoe Store,Smoke Shop,Snack Place,Soba Restaurant,Social Club,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Storage Facility,Street Art,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tapas Restaurant,Tattoo Parlor,Tea Room,Temple,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Thrift / Vintage Store,Tiki Bar,Toy / Game Store,Trail,Tree,Turkish Restaurant,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Vietnamese Restaurant,Watch Shop,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,NEIGHBORHOOD_NAME,MODIFIED_ZCTA,COVID_DEATH_RATE
0,10001,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.025641,0.0,0.00,0.000000,0.051282,0.025641,0.0,0.0,0.0,0.0,0.0,0.0,0.025641,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.00,0.076923,